In [3]:
!git clone https://github.com/charlieardern/transformers-from-scratch

fatal: destination path 'transformers-from-scratch' already exists and is not an empty directory.


In [8]:
%cd transformers-from-scratch/vanilla-transformer

/kaggle/working/transformers-from-scratch/vanilla-transformer


# Building a Transformer from Scratch
Charlie Ardern <br>
<hr>

*This is not yet functional. Updates to come soon.*

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

The first objective is to turn the $X$ matrix into a set of $Q,K,V$ matrices for each head. This can be implemented efficiently with one matrix operation by using `nn.Linear`. We construct a large matrix, of dimension `(batch_size, sequence_length, 3*d*h)`. This will have all required matrices concatenated along `dim=2`.
- Note that $d$ is the dimension of each token, $h$ is the number of heads and the 3 comes about as we are consturcting 3 matrices (Q,K,V) for each head. (See the image below)
- Note that `nn.Linear` is designed to compute operations on a vector so it will perform the same operation on each token of each batch.<br>

Here we see the operation carried out by the linear layer:
<br>

<br>
And looking more closely at the output:
<br>



**The transformer encoder will consist of these key stages:**
1. Positional encoding
2. Multi-head attention
3. Add + layer norm
4. MLP
5. Add + layer norm

In [ ]:
class MHA(nn.Module):
    
    def __init__(self, d, h):
        super().__init__()
        assert d % h == 0, "Dimension of embeddings must be divisible by number of heads"
        self.d = d
        self.h = h    
        self.X_to_QKV = nn.Linear(d, 3*d)
        self.stacked_heads_to_X = nn.Linear(d, d)
        self._reset_parameters()
        
    def _reset_parameters(self):
        nn.init.xavier_uniform_(self.X_to_QKV.weight)
        nn.init.xavier_uniform_(self.stacked_heads_to_X.weight)
        self.X_to_QKV.bias.data.fill_(0)
        self.stacked_heads_to_X.bias.data.fill_(0)
        
    def forward(self, x):
        b, n, d = x.size()
        QKV = self.X_to_QKV(X)
        QKV = QKV.reshape(b, n, h, 3*d/h)
        QKV = QKV.permute(0, 2, 1, 3)
        Q, K, V = QKV.chunk(3, dim=-1)
        
        # Scaled dot product attention:
        logits = torch.matmul(Q,K.transpose(-2,-1))/torch.sqrt(d/h)
        #can implement masking here later
        attention = F.softmax(logits, dim=-1)
        head_vals = torch.matmul(attention, V)
        
        head_vals = head_vals.permute(0, 2, 1, 3)
        stacked_heads = heav_vals.reshape(b, n, d)
        X = self.stacked_heads_to_X(stacked_heads)
        return X